Завдання 4

Аналогічно попередньому завданню, в директорії task-4 знаходиться файл output.txt що містить:

* відкритий ключ (значення модуля n та відкритої експоненти e);
* шифротекст ct, згенерованим цим відкритим ключем.

Секретного ключа для розшифрування немає.

Поруч є файл modulus_inutilis.py з кодом, який був використаний для генерування цього шифротексту. З коду може бути видно, що для зашифрування використовувалось базове перетворення RSA без доповнення.

Зверніть увагу на велике значення модуля, так значення експоненти e = 3. Подумайте як можна відновити повідомлення, у випадку коли отриманий шифротекст ніколи не перевищив значення модуля?

Дешифруйте повідомлення без використання секретного ключа.

Додаткова інформація

  * Для криптографічних операцій користуйтесь бібліотекою pycryptodome.

pip install pycryptodome

* Для перетворення строки з числовим значенням у змінну і навпаки, можете користуватись функціями:

from Crypto.Util.number import bytes_to_long, long_to_bytes

* Дешифроване повідомлення матиме формат флагу з платформи CryptoHack: crypto{xxxxxxxxx}.

Формат виконання

Завантажте код використаний для дешифрування у ваш репозиторій з домашніми завданнями. У текстовому файлі також прикріпіть отриманий флаг.

In [21]:
!pip install pycryptodome
!pip install gmpy2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 17.4 MB/s eta 0:00:00


In [2]:
from Crypto.Util.number import bytes_to_long, long_to_bytes

Перевіримо наданий алгоритм шифрування (дещо модифікувавши його для уникнення зайвих помилок при визначенні оберненого до e за модулем $\phi$)

In [17]:
#!/usr/bin/env python3

from Crypto.Util.number import getPrime, inverse, bytes_to_long, long_to_bytes, GCD

e = 3
d = -1

while d == -1:
    p = getPrime(1024)
    print('p=', p)
    q = getPrime(1024)
    print('q=', q)
    phi = (p - 1) * (q - 1)

    if GCD(e, phi) == 1:  # Перевірка, чи e і phi взаємно прості
        d = inverse(e, phi)


n = p * q

flag = b"crypto{xxxxxxxxxxxx}"
pt = bytes_to_long(flag)
ct = pow(pt, e, n)

print(f"n = {n}")
print(f"e = {e}")
print(f"ct = {ct}")

pt = pow(ct, d, n)
decrypted = long_to_bytes(pt)
assert decrypted == flag

p= 170611947913261179018804405939387391922479415857065354137466613596925759298707150854697023922030359921579583820939937068919258108334017757427073371553890283883252511453886633333703119612882600942880432886912456960513868099603477485062101673453453093427683253787922488307984094463229806725739712130233444255649
q= 121237847951666742310690895307489254171175771444221043888338376989200944437018478366592383295597153171532232695856048859168549572678259631059662839603952837293563961320862864392582259681897055131581881607241288469705549037694070085457851040172396927030405000809506069211660836699354652295951870534037798721389
p= 100367039144271565113267339987799406302858850997586300213019636909884354032821127780045041620314591408030961419616468881870594981996685334426105193349902830512324026002293432419971069849608972512963944938313750683702016969546595374174938802868755822361463111171569599325903728275609213692297151799576599951489
q= 1020490893659743771997633150847125235728857568736332416895

In [23]:
#Input data
n = 30460296206281253605887131667441042408833105116654370140736576080711297109384941590369941855116386695157474206375705248890458232777575365270780855265861075198881090190505284920581410885950363830131451127387018904728639607372668753109249046707840464876881594185896506371262697868257217488062754637361594352910022190227237953540282162231147699265142164623465337280610190892470279654386272723760887111753067292988287956381022028725288845603024605833650847697724636088418782911705757980221361510892370739837402705040814150778298018509675199917931423568797098139493145394232981571448400646089157848498064505852923746440139
e = 3
ct = 183001753190025751114220069887230720857448492282044619321040127443487542179613757444809112210217896463899655491288132907560322811734646233820773


Отже, маємо інформацію, що c << n , відповідно, якщо значення $c = m^e < n$, то при e=3 отримуємо, що  $m = \sqrt[3]{c}$.

Перевіримо це, взявши цілочисельний корінь 3-степеня з c

In [26]:
from gmpy2 import iroot
from Crypto.Util.number import long_to_bytes


# Знаходимо кубічний корінь
pt, exact = iroot(ct, 3)  # Обчислюємо цілий кубічний корінь

# Якщо корінь точний, перетворюємо у байтовий рядок
if exact:
    flag = long_to_bytes(int(pt))
    print("Розшифроване повідомлення:", flag.decode())
else:
    print("Точний кубічний корінь не знайдено" )

Розшифроване повідомлення: crypto{robot_dreams}


Завершальний етап: збереження даних

In [29]:
path = "/content/drive/MyDrive/robot_dreams_cryptography/HW6/"

with open(path + "result_task4.txt", "w") as outlet_file:
    outlet_file.write(flag.decode())